In [129]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
hacktivist_messages = pd.read_csv("data/hacktivist_messages.csv", sep=";")
hacktivist_messages = hacktivist_messages.dropna()

In [133]:
from flair.data import Sentence
from flair.models import SequenceTagger
import math

# load tagger
tagger = SequenceTagger.load("flair/ner-english-fast")

tagged_texts = []
for row in tqdm(hacktivist_messages.iterrows(), total=hacktivist_messages.shape[0]):
    text = Sentence(row[1]["Text"])
    tagger.predict(text)
    tagged_texts.append(text)

hacktivist_messages["NER"] = tagged_texts

2024-09-04 12:51:36,393 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


100%|██████████| 2995/2995 [17:09<00:00,  2.91it/s]


In [183]:
tagger_ontonotes = SequenceTagger.load("ner-ontonotes-fast")

tagged_texts_ontonotes = []
for row in tqdm(hacktivist_messages.iterrows(), total=hacktivist_messages.shape[0]):
    text = Sentence(row[1]["Text"])
    tagger_ontonotes.predict(text)
    tagged_texts_ontonotes.append(text)

hacktivist_messages["NER_onto"] = tagged_texts_ontonotes

pytorch_model.bin: 100%|██████████| 1.30G/1.30G [01:36<00:00, 13.4MB/s]


2024-09-04 13:41:36,505 SequenceTagger predicts: Dictionary with 75 tags: O, S-PERSON, B-PERSON, E-PERSON, I-PERSON, S-GPE, B-GPE, E-GPE, I-GPE, S-ORG, B-ORG, E-ORG, I-ORG, S-DATE, B-DATE, E-DATE, I-DATE, S-CARDINAL, B-CARDINAL, E-CARDINAL, I-CARDINAL, S-NORP, B-NORP, E-NORP, I-NORP, S-MONEY, B-MONEY, E-MONEY, I-MONEY, S-PERCENT, B-PERCENT, E-PERCENT, I-PERCENT, S-ORDINAL, B-ORDINAL, E-ORDINAL, I-ORDINAL, S-LOC, B-LOC, E-LOC, I-LOC, S-TIME, B-TIME, E-TIME, I-TIME, S-WORK_OF_ART, B-WORK_OF_ART, E-WORK_OF_ART, I-WORK_OF_ART, S-FAC


 11%|█         | 330/2995 [01:20<10:47,  4.12it/s]


KeyboardInterrupt: 

In [174]:
message_ids = []
labels = []
label_texts = []

for row in hacktivist_messages.iterrows():
    sentence = row[1]["NER"]
    message_id = row[1]["Message Id"]
    for label in sentence.get_labels():
        message_ids.append(message_id)
        labels.append(label.value)
        label_texts.append(label.data_point.text)

ner_tagged_data = pd.DataFrame({
    "message_ids": message_ids, 
    "text": label_texts,
    "label":labels
})

In [ ]:
hacktivist_messages

In [182]:
ner_tagged_data[ner_tagged_data["label"] == 'LOC']["text"].unique()


array(['Poland', 'Russia', 'Belarus', 'USA', 'Lithuania', 'Latvia',
       'Wągrowiec', 'Krotoshin', 'Lobez', 'Estonia', 'Warmia',
       'Mazury Voivodeship', 'Pomeranian Voivodeship Court of Poland',
       'Kuyavian-Pomeranian Voivodeship of Poland', 'Motherland',
       'Pomeranian Voivodeship of Poland', 'Swietokrzyski Voivodeship',
       'Limanow', 'Sieradz', 'Wlodawa', 'Yawor', 'Nowy Dwor Mazowiecki',
       'Sweden', 'Germany', 'Hungary', 'Brodnitsa', 'Krakow Nowa Huta',
       'Glogow', 'Sroda-Slianska', 'Riga', 'State Training Center',
       'Gdynia', 'Tallinn', 'Kuressaare', 'Ukraine', 'Austria', 'Italy',
       'France', 'Switzerland', 'Western Europe', 'Rzeszow', 'Ventspils',
       'Warsaw', 'Denmark', 'Jyskebank', 'Czech Republic', 'Kiev',
       'Bulgaria', 'Europe', 'Dnieper', 'Dniester', 'US', 'Washington',
       'Europe.And', 'Kyiv', 'Zaporizhzhya', 'Zhytomyr', 'Stockholm',
       'Ukraine.', 'Paris', 'UK', 'Norway', 'Finland', 'Iceland',
       'Moscow', 'Vilnius

In [ ]:
ner_tagged_data.to_parquet("data/ner_tagged_data.parquet")